In [1]:
library('SelectSim')
library('tidyverse')
library('tictoc')

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] tictoc_1.2        lubridate_1.9.3   forcats_1.0.0     stringr_1.5.1    
 [5] dplyr_1.1.4       purrr_1.0.2       readr_2.1.5       tidyr_1.3.0      
 [9] tibble_3.2.1      ggplot2_3.4.4     tidyverse_2.0.0   SelectSim_0.0.1.3

## Merge TCGA, MSK, DFCI results

In [3]:
all_result_merged_msk<- readRDS(file='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/results/all_merged_result_msk_v15.rds')
all_result_merged_dfci<- readRDS(file='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/results/all_merged_result_dfci_v15.rds')
all_result_merged_tcga<- readRDS(file='/mnt/ptemp/arvind/catalogue_work_update/data/results/tcga_merged_result.rds')

In [4]:
head(all_result_merged_msk)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,nES,mean_r_nES,nFDR,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,TP53,PIK3CA,TP53 - PIK3CA,12075,3970,0.43396226,0.14267745,1429,1055.34778,3243,⋯,-296.1027,-13.019157,0,16045,0,ME,TRUE,MSK Primary,all,Pan-can
2,KRAS,EGFR,KRAS - EGFR,5622,1259,0.20204852,0.04524708,36,15.12377,1032,⋯,-192.4016,-6.245916,0,6881,0,ME,TRUE,MSK Primary,all,Pan-can
3,PTEN,TP53,PTEN - TP53,2469,12075,0.08873315,0.43396226,695,468.87573,1655,⋯,-172.3259,-9.869776,0,14544,0,ME,TRUE,MSK Primary,all,Pan-can
4,ARID1A,TP53,ARID1A - TP53,2866,12075,0.10300090,0.43396226,988,630.94088,2150,⋯,-168.1535,-10.759211,0,14941,0,ME,TRUE,MSK Primary,all,Pan-can
5,KRAS,TP53,KRAS - TP53,5622,12075,0.20204852,0.43396226,2773,2418.48500,5013,⋯,-132.9546,-15.025226,0,17697,0,ME,TRUE,MSK Primary,all,Pan-can
6,TP53,APC,TP53 - APC,12075,3486,0.43396226,0.12528302,2302,1905.36483,3239,⋯,130.2265,12.274751,0,15561,0,CO,TRUE,MSK Primary,all,Pan-can


In [5]:
head(all_result_merged_tcga)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,wFDR,nES,mean_r_nES,nFDR,cum_freq,nFDR2,type,FDR,datatype,cohort
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>
1,PIK3CA,TP53,PIK3CA - TP53,1136,3030,0.12508258,0.33362695,357,317.529713,669,⋯,0,-46.41800,-7.357413,0,4166,0,ME,TRUE,Pan-can,TCGA
2,BRAF,TP53,BRAF - TP53,611,3030,0.06727593,0.33362695,82,53.397547,180,⋯,0,-44.24852,-4.784206,0,3641,0,ME,TRUE,Pan-can,TCGA
3,TP53,ARID1A,TP53 - ARID1A,3030,571,0.33362695,0.06287161,147,115.192451,371,⋯,0,-34.12723,-4.990387,0,3601,0,ME,TRUE,Pan-can,TCGA
4,BRAF,NRAS,BRAF - NRAS,611,240,0.06727593,0.02642590,13,6.352379,191,⋯,0,-31.75002,-2.704795,0,851,0,ME,TRUE,Pan-can,TCGA
5,CDKN2A,TP53,CDKN2A - TP53,334,3030,0.03677604,0.33362695,225,202.011705,334,⋯,0,29.37771,4.329415,0,3364,0,CO,TRUE,Pan-can,TCGA
6,PTEN,TP53,PTEN - TP53,635,3030,0.06991852,0.33362695,158,133.797790,385,⋯,0,-28.99619,-5.350602,0,3665,0,ME,TRUE,Pan-can,TCGA


In [6]:
all_result_merged_tcga$Panel <- 'WES'

In [7]:
all_result_merged_tcga$Cohort<-all_result_merged_tcga$cohort
all_result_merged_tcga$Tumor_run<-all_result_merged_tcga$datatype

In [8]:
all_merged_result_v15 <- rbind(all_result_merged_tcga[,colnames(all_result_merged_msk)],all_result_merged_msk,all_result_merged_dfci)

In [9]:
dim(all_merged_result_v15)

[1] 2869359      25

In [10]:
head(all_merged_result_v15)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,nES,mean_r_nES,nFDR,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,PIK3CA,TP53,PIK3CA - TP53,1136,3030,0.12508258,0.33362695,357,317.529713,669,⋯,-46.41800,-7.357413,0,4166,0,ME,TRUE,TCGA,WES,Pan-can
2,BRAF,TP53,BRAF - TP53,611,3030,0.06727593,0.33362695,82,53.397547,180,⋯,-44.24852,-4.784206,0,3641,0,ME,TRUE,TCGA,WES,Pan-can
3,TP53,ARID1A,TP53 - ARID1A,3030,571,0.33362695,0.06287161,147,115.192451,371,⋯,-34.12723,-4.990387,0,3601,0,ME,TRUE,TCGA,WES,Pan-can
4,BRAF,NRAS,BRAF - NRAS,611,240,0.06727593,0.02642590,13,6.352379,191,⋯,-31.75002,-2.704795,0,851,0,ME,TRUE,TCGA,WES,Pan-can
5,CDKN2A,TP53,CDKN2A - TP53,334,3030,0.03677604,0.33362695,225,202.011705,334,⋯,29.37771,4.329415,0,3364,0,CO,TRUE,TCGA,WES,Pan-can
6,PTEN,TP53,PTEN - TP53,635,3030,0.06991852,0.33362695,158,133.797790,385,⋯,-28.99619,-5.350602,0,3665,0,ME,TRUE,TCGA,WES,Pan-can


In [11]:
head(all_merged_result_v15)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,nES,mean_r_nES,nFDR,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,PIK3CA,TP53,PIK3CA - TP53,1136,3030,0.12508258,0.33362695,357,317.529713,669,⋯,-46.41800,-7.357413,0,4166,0,ME,TRUE,TCGA,WES,Pan-can
2,BRAF,TP53,BRAF - TP53,611,3030,0.06727593,0.33362695,82,53.397547,180,⋯,-44.24852,-4.784206,0,3641,0,ME,TRUE,TCGA,WES,Pan-can
3,TP53,ARID1A,TP53 - ARID1A,3030,571,0.33362695,0.06287161,147,115.192451,371,⋯,-34.12723,-4.990387,0,3601,0,ME,TRUE,TCGA,WES,Pan-can
4,BRAF,NRAS,BRAF - NRAS,611,240,0.06727593,0.02642590,13,6.352379,191,⋯,-31.75002,-2.704795,0,851,0,ME,TRUE,TCGA,WES,Pan-can
5,CDKN2A,TP53,CDKN2A - TP53,334,3030,0.03677604,0.33362695,225,202.011705,334,⋯,29.37771,4.329415,0,3364,0,CO,TRUE,TCGA,WES,Pan-can
6,PTEN,TP53,PTEN - TP53,635,3030,0.06991852,0.33362695,158,133.797790,385,⋯,-28.99619,-5.350602,0,3665,0,ME,TRUE,TCGA,WES,Pan-can


In [12]:
tumor_mapped<-readRDS(file='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/data/tcga_tumor_run_mapping.rds')

In [14]:
all_merged_result<-all_merged_result_v15 %>% mutate(Mapped_Tumor_run = case_when(Tumor_run %in% names(tumor_mapped) ~ tumor_mapped[Tumor_run],
                                                                   TRUE ~ Tumor_run))

In [15]:
head(all_merged_result)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,mean_r_nES,nFDR,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run,Mapped_Tumor_run
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
1,PIK3CA,TP53,PIK3CA - TP53,1136,3030,0.12508258,0.33362695,357,317.529713,669,⋯,-7.357413,0,4166,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can
2,BRAF,TP53,BRAF - TP53,611,3030,0.06727593,0.33362695,82,53.397547,180,⋯,-4.784206,0,3641,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can
3,TP53,ARID1A,TP53 - ARID1A,3030,571,0.33362695,0.06287161,147,115.192451,371,⋯,-4.990387,0,3601,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can
4,BRAF,NRAS,BRAF - NRAS,611,240,0.06727593,0.02642590,13,6.352379,191,⋯,-2.704795,0,851,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can
5,CDKN2A,TP53,CDKN2A - TP53,334,3030,0.03677604,0.33362695,225,202.011705,334,⋯,4.329415,0,3364,0,CO,TRUE,TCGA,WES,Pan-can,Pan-can
6,PTEN,TP53,PTEN - TP53,635,3030,0.06991852,0.33362695,158,133.797790,385,⋯,-5.350602,0,3665,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can


In [16]:
unique(all_merged_result$Mapped_Tumor_run)

[1] "Pan-can"                              
 [2] "no_mapping"                           
 [3] "bladder_cancer"                       
 [4] "breast_cancer"                        
 [5] "cervical_cancer"                      
 [6] "billary_tract_cancer"                 
 [7] "colorectal_cancer"                    
 [8] "mature_b_cell_neoplasms"              
 [9] "esophagogastric_cancer"               
[10] "glioma"                               
[11] "head_and_neck_cancer"                 
[12] "renal_cell_carcinoma"                 
[13] "leukemia"                             
[14] "liver_cancer"                         
[15] "non_small_cell_lung_cancer"           
[16] "mesothelioma"                         
[17] "ovarian_cancer"                       
[18] "pancreatic_cancer"                    
[19] "prostate_cancer"                      
[20] "soft_tissue_sarcoma"                  
[21] "melanoma"                             
[22] "germ_cell_tumor"                      
[23] "thyroid_cancer"                       
[24] "endometrial_cancer"                   
[25] "salivary_gland_cancer"                
[26] "anal_cancer"                          
[27] "appendiceal_cancer"                   
[28] "b_lymphoblastic_leukemia_lymphoma"    
[29] "skin_cancer,_non_melanoma"            
[30] "gastrointestinal_stromal_tumor"       
[31] "myeloproliferative_neoplasms"         
[32] "peripheral_nervous_system"            
[33] "bone_cancer"                          
[34] "small_cell_lung_cancer"               
[35] "uterine_sarcoma"                      
[36] "cns_cancer"                           
[37] "gastrointestinal_neuroendocrine_tumor"
[38] "sellar_tumor"

In [17]:
all_merged_result %>% count(Mapped_Tumor_run)

Mapped_Tumor_run,n
<chr>,<int>
Pan-can,445055
anal_cancer,13538
appendiceal_cancer,6522
b_lymphoblastic_leukemia_lymphoma,5968
billary_tract_cancer,55682
bladder_cancer,175009
bone_cancer,735
breast_cancer,179267
cervical_cancer,24046


In [18]:
str(all_merged_result)

'data.frame':	2869359 obs. of  26 variables:
 $ SFE_1           : chr  "PIK3CA" "BRAF" "TP53" "BRAF" ...
 $ SFE_2           : chr  "TP53" "TP53" "ARID1A" "NRAS" ...
 $ name            : chr  "PIK3CA - TP53" "BRAF - TP53" "TP53 - ARID1A" "BRAF - NRAS" ...
 $ support_1       : num  1136 611 3030 611 334 ...
 $ support_2       : num  3030 3030 571 240 3030 3030 3030 286 571 684 ...
 $ freq_1          : num  0.1251 0.0673 0.3336 0.0673 0.0368 ...
 $ freq_2          : num  0.3336 0.3336 0.0629 0.0264 0.3336 ...
 $ overlap         : num  357 82 147 13 225 158 50 162 201 16 ...
 $ w_overlap       : num  317.53 53.4 115.19 6.35 202.01 ...
 $ max_overlap     : num  669 180 371 191 334 385 229 193 359 125 ...
 $ freq_overlap    : num  0.5336 0.4556 0.3962 0.0681 0.6737 ...
 $ r_overlap       : num  485.2 181.9 243.4 99.8 199.3 ...
 $ w_r_overlap     : num  393.6 122.7 170.5 55.1 154.3 ...
 $ wES             : num  -53.8 -49 -39.1 -34.5 33.7 ...
 $ wFDR            : num  0 0 0 0 0 ...
 $ nES     

In [ ]:
saveRDS(all_merged_result,file='../results/all_merged_results_v15.rds')